- GPU를 사용하기 위해 google colab를 사용하여 구현하였습니다.
- google colab 파일의 확장명은 ipynb인데, 원래 코드의 가독성 때문에 각 기능들의 파일을 만들고 거기에 각 기능에 해당하는 코드를 만드려고 했는데 ipynb의 파일은 불러올 수 없다고 해서 (ex. from!!! import!!!! 여기서 from의 !!!이 ipynb이면 못 가져옴(*!!!: 파일명, !!!!: !!!코드에 있는 클래스나 함수)) 한 파일에 모든 기능과 백엔드와 ai를 연결하는 fast api도 구현하였습니다.

In [ ]:
# 필요한 패키지 설치
!pip install fastapi uvicorn nest-asyncio
!pip install --upgrade openai google-api-python-client
!pip install transformers torch tqdm httpx konlpy nltk langid langdetect
!pip install fugashi[unidic-lite] jieba pythainlp
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 87.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.84.0
    Uninstalling openai-1.84.0:
      Successfully uninstalled openai-1.84.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.171.0
    Uninstalling google-api-python-client-2.171.0:
      Successfully uninstalled google-api-python-client-2.171.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6

In [ ]:
import os
import asyncio
import httpx
import random
import re
import time
import warnings
from collections import Counter
from typing import Dict, List, Optional

# FastAPI 관련
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel,Field
import uvicorn
import nest_asyncio

# AI/ML 관련
from openai import OpenAI
from transformers import pipeline
from tqdm import tqdm
import nltk
import langid
from langdetect import detect, DetectorFactory

# 한국어/다국어 처리
from fugashi import Tagger
from konlpy.tag import Okt
import jieba
from pythainlp.tokenize import word_tokenize as thai_tokenize
from pythainlp.corpus.common import thai_stopwords
from pythainlp.util import normalize
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# ngrok
from pyngrok import ngrok

# 설정
warnings.filterwarnings("ignore")
nest_asyncio.apply()
DetectorFactory.seed = 0

In [ ]:
ngrok.set_auth_token("NGROK_TOKEN") # 임시
ngrok.kill() # 기존 포트 연결 해제

# 8000 포트에 ngrok 터널 열기
public_url = ngrok.connect(8000)
print(f"FastAPI 주소: {public_url}/analyze")

# API 키 설정 (환경변수에서 가져오기)
# OpenAI api 는 "전체 댓글 요약"에서 사용하고 DeepL api 는 "감정분석"할 때 사용됨
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY") # 임시
DEEPL_API_KEY = os.getenv("DEEPL_API_KEY", "DEEPL_API_KEY") # 임시

FastAPI 주소: NgrokTunnel: "https://9e81-35-224-42-74.ngrok-free.app" -> "http://localhost:8000"/analyze


In [ ]:
# NLTK 다운로드
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt_tab')
    nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# 클라이언트 초기화
client = OpenAI(api_key=OPENAI_API_KEY)

# 모델 초기화
# 감정 분석할 때 사용하는 모델
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    top_k=1,
    truncation=True,
    batch_size=64,
    max_length=128
)

# 의심 댓글 분류할 때 사용하는 모델
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 형태소 분석기 초기화
okt = Okt()
tagger = Tagger()

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# 불용어 설정
supported_langs = stopwords.fileids()
multi_lang_stopwords = set()
for lang in supported_langs:
    multi_lang_stopwords.update(stopwords.words(lang))

# 불용어 처리
# "키워드 추출"할 때 의미 없는 단어가 추출되면 안되니까 커스텀으로 지정해줌
# 물론 불용어 지원해주는 언어들은 라이브러리도 사용했음
stop_words_ko = {'그냥', '정말', '진짜', '너무', '영상', 'ㅋㅋ', 'ㅠㅠ', '다시','보고','제일', '여기'}
custom_stopwords_en = {'im', 'ive', 'youre', 'also', 'like', 'really', 'u', 'so', 'much', 'know', 'thank', 'just', 'watching', 'feel', 'video', 'videos', 'channel', 'love', 'life', 'one'}
custom_stopwords_fr = {'cest', 'jai', 'ça', 'trop', 'vraiment', 'vidéo', 'vidéos', 'très', 'super', 'bien', 'grave', 'cool', 'genre', 'juste', 'encore', 'ouais', 'ouai', 'ouaip', 'la', 'le', 'les', 'des', 'un', 'une', 'fait', 'tellement', 'tout'}
custom_stopwords_ru = {'просто', 'очень', 'реально', 'спасибо', 'пожалуйста', 'вообще', 'супер', 'нормально', 'да', 'нет', 'ага', 'уже', 'видео', 'ролик', 'это', 'такое', 'все', 'ничего', 'что', 'когда', 'как', 'тут', 'там', 'где', 'чем', 'ещё'}
custom_stopwords_ja = {'です', 'ます', 'ちゃんと', 'てる', 'こと', 'もの', 'それ', 'これ', 'あれ', 'のに', 'よう', 'そう', 'みんな', 'とか', 'から', 'でも', 'ね', 'なぁ', 'ん', 'おん', 'ちゃん', 'ほんと', 'マジ', 'すごい', 'けど', 'すぎ', 'すぎる', 'にょ', 'また', 'やっ', 'まし'}
custom_stopwords_th = {'กน', 'คล', 'ชว', 'อย', 'แล', 'เพ', 'ได', 'โมง', 'ทำ', 'นา', 'ผม', 'พเนท', 'สน', 'อน', 'เห', 'ไม','ให'}
stop_words_th = set(thai_stopwords())

# 커스텀한 불용어 + 라이브러리를 사용한 불용어 합침
all_stopwords = multi_lang_stopwords.union(
    stop_words_ko, custom_stopwords_en, custom_stopwords_fr,
    custom_stopwords_ru, custom_stopwords_ja, stop_words_th, custom_stopwords_th
)

# 언어 매핑
# 라이브러리를 사용해서 언어 비율 분석할 때 예를 들면 결과가 "af": 70%,"ca":20%... 이런 식으로 처음에 나왔는데 프론트엔드에서 이걸 한국어로 보내달라고 해서
# 라이브러리에서 나올 수 있는 모든 언어들을 찾은 후 af, sq, .. 이것들이 뭔지 한국어로 매핑시킴
LANGUAGE_NAME_MAP = {
    "af": "아프리칸스어", "sq": "알바니아어", "ar": "아랍어", "az": "아제르바이잔어",
    "be": "벨라루스어", "bg": "불가리아어", "bn": "벵골어", "ca": "카탈루냐어",
    "cs": "체코어", "cy": "웨일스어", "da": "덴마크어", "de": "독일어",
    "el": "그리스어", "en": "영어", "es": "스페인어", "et": "에스토니아어",
    "fa": "페르시아어", "fi": "핀란드어", "fr": "프랑스어", "gu": "구자라트어",
    "he": "히브리어", "hi": "힌디어", "hr": "크로아티아어", "hu": "헝가리어",
    "id": "인도네시아어", "is": "아이슬란드어", "it": "이탈리아어", "ja": "일본어",
    "jv": "자바어", "ka": "조지아어", "kk": "카자흐어", "km": "크메르어",
    "kn": "칸나다어", "ko": "한국어", "lt": "리투아니아어", "lv": "라트비아어",
    "mk": "마케도니아어", "ml": "말라얄람어", "mr": "마라티어", "ms": "말레이어",
    "my": "버마어", "ne": "네팔어", "nl": "네덜란드어", "no": "노르웨이어",
    "pa": "펀자브어", "pl": "폴란드어", "pt": "포르투갈어", "ro": "루마니아어",
    "ru": "러시아어", "sk": "슬로바키아어", "sl": "슬로베니아어", "sv": "스웨덴어",
    "sw": "스와힐리어", "ta": "타밀어", "te": "텔루구어", "th": "태국어",
    "tl": "타갈로그어", "tr": "터키어", "uk": "우크라이나어", "ur": "우르두어",
    "vi": "베트남어", "zh-cn": "중국어(간체)", "zh-tw": "중국어(번체)"
}

In [ ]:
# Pydantic 모델 정의
class AnalysisRequest(BaseModel):
    videoId: str = Field(alias="videoId")
    comments: Dict[str, str] = Field(alias="comments")

    class Config:
        allow_population_by_field_name = True

class AnalysisResponse(BaseModel):
    videoId: Optional[int] = None
    apiVideoId: str
    summation: str
    isWarning: bool
    keywords: List[str]
    sentimentComments: Dict[str, str]  # Map<apiCommentId, SentimentType>
    languageRatio: Dict[str, float]
    sentimentRatio: Dict[str, float]

In [ ]:
# 유틸리티 함수들

def clean_comment(text: str) -> str:
    """댓글 전처리"""
    text = re.sub(r"[^\w\s.,!?ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z]", "", text)
    text = re.sub(r"http\S+|www\.\S+", "", text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)
    text = text.replace('\n', ' ').replace('\r', ' ')
    return text.strip()

def is_english(text: str) -> bool:
    """영어 여부 판단"""
    cleaned = re.sub(r"[^\w\s.,!?\'\"-]", '', text)
    return re.fullmatch(r'[A-Za-z0-9\s\.,;:\'\"!?()\[\]{}@#$%^&*_\-=+/<>|~]+', cleaned) is not None

# 감정 분석 할 때 사용하는 모델이 영어 기반이어서 DeepL api 를 사용해서 번역한 후 모델에 입력하였음
# 따라서 이 기능의 결과가 나올 때 시간이 다른 기능들보다 오래걸렸음
async def translate_text_async(client, text: str, api_key: str, target_lang: str = "EN") -> str:
    """DeepL 비동기 번역"""
    url = "https://api.deepl.com/v2/translate"
    data = {
        "auth_key": api_key,
        "text": text,
        "target_lang": target_lang,
    }
    try:
        await asyncio.sleep(random.uniform(0.05, 0.15))
        response = await client.post(url, data=data, timeout=10.0)
        response.raise_for_status()
        return response.json()["translations"][0]["text"]
    except Exception as e:
        return f"[ERROR: {str(e)}]"

def detect_language(text: str) -> str:
    """언어 감지"""
    lang, _ = langid.classify(text)
    return lang

def mixed_tokenizer(text: str) -> List[str]:
    """다국어 토크나이저"""
    lang = detect_language(text)

    if lang == 'ko':
        tokens = okt.nouns(text)
        tokens = [t for t in tokens if t not in all_stopwords and len(t) > 1]
    elif lang == 'ja':
        tokens = [word.surface for word in tagger(text) if word.surface not in all_stopwords and len(word.surface) > 1]
    elif lang in ('zh', 'zh-cn', 'zh-tw'):
        tokens = [t for t in jieba.cut(text) if t not in all_stopwords and len(t.strip()) > 1]
    elif lang == 'th':
        tokens = []
        for t in thai_tokenize(text, engine="newmm", keep_whitespace=False):
            t_norm = normalize(t.strip())
            if len(t_norm) >= 2 and not t_norm.isnumeric() and t_norm not in all_stopwords:
                tokens.append(t_norm)
    else:
        tokens = [w.lower() for w in word_tokenize(text) if w.isalpha() and w.lower() not in all_stopwords]

    return tokens

In [ ]:
# 분석 함수들

# 전체 댓글 요약할 때 사용한 함수
# OpenAI api 를 사용하기 때문에 gpt한테 부탁할 프롬프트 작성함
# 우리 웹사이트는 한국어 기반으로 구성되어있기 때문에 영어로 된 비디오라도 요약을 한국어로 부탁함
def summarize_comments_with_gpt(comments: List[str]) -> str:
    """GPT를 이용한 댓글 요약"""
    joined = "\n".join(comments)
    prompt = f"""당신은 텍스트 요약 전문가입니다.
다음은 어떤 유튜브 영상에 달린 다양한 댓글들입니다.
이 댓글들의 전체 분위기와 주요 논점만 2문장 이내로 자연스럽고 간결하게 요약해주세요.
중복된 의견은 묶어서 정리하고, 인상적인 반응은 반영해 주세요.
그리고 한국어로 나타내주세요.
댓글 목록:
{joined}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=500
    )

    return response.choices[0].message.content.strip()

# 댓글 감정 분석할 때 사용한 함수
async def analyze_sentiment_async(comments_dict: Dict[str, str]) -> tuple:
    """댓글 감정 분석"""
    async with httpx.AsyncClient() as http_client:
        processed_comments = []

        # 여기서 사용한 모델이 영어 기반이어서 영어이면 그대로, 영어가 아니면 DeepL api 를 사용해서 번역하도록 구성했음
        for comment_id, comment_text in comments_dict.items():
            try:
                if is_english(comment_text):
                    translated = comment_text
                else:
                    translated = await translate_text_async(http_client, comment_text, DEEPL_API_KEY)

                processed_comments.append({
                    "id": comment_id,
                    "original": comment_text,
                    "translated": translated
                })
            except Exception as e:
                processed_comments.append({
                    "id": comment_id,
                    "original": comment_text,
                    "translated": comment_text
                })

    # 감정 분석 실행
    texts = [c["translated"] for c in processed_comments]
    sentiments = sentiment_analyzer(texts, batch_size=16)

    sentiment_dict = {}
    for i, s in enumerate(sentiments):
        label = s[0]["label"]
        sentiment = {
            "LABEL_0": "NEGATIVE",
            "LABEL_2": "POSITIVE",
        }.get(label, "OTHER")
        comment_id = processed_comments[i]["id"]
        sentiment_dict[comment_id] = sentiment

    # 감정 비율 계산
    # 감정 분석을 한 후 positive, negative, other에 대한 비율을 구함
    counts = Counter(sentiment_dict.values())
    total = sum(counts.values())

    sentiment_ratio = {
        "positive": round(counts.get("POSITIVE", 0) / total * 100, 2),
        "other": round(counts.get("OTHER", 0) / total * 100, 2),
        "negative": round(counts.get("NEGATIVE", 0) / total * 100, 2),
    }

    return sentiment_dict, sentiment_ratio

# 키워드를 추출할 때 사용하는 함수
# 텍스트 마이닝 수업 때 배운 TF-IDF를 사용함
def extract_keywords_tfidf(comments: List[str], top_n: int = 5) -> List[str]:
    """TF-IDF 키워드 추출"""
    try:
        vectorizer = TfidfVectorizer(
            tokenizer=mixed_tokenizer,
            analyzer='word',
            lowercase=False,
            max_features=top_n
        )
        vectorizer.fit_transform(comments)
        return list(vectorizer.get_feature_names_out())
    except:
        return []

# 언어 비율을 분석할 때 사용한 함수
# langdetect 라이브러리를 사용함
def detect_languages(comments: List[str]) -> Dict[str, float]:
    """언어 비율 분석"""
    lang_list = []
    for comment in comments:
        if len(comment.strip()) < 3:
            continue
        try:
            lang = detect(comment)
            lang_list.append(lang)
        except:
            continue

    if not lang_list:
        return {}

    lang_count = Counter(lang_list)
    total = sum(lang_count.values())

    lang_ratio = {}
    for lang_code, count in lang_count.items():
        if count > 0:
            lang_name = LANGUAGE_NAME_MAP.get(lang_code, f"기타({lang_code})")
            ratio = round((count / total) * 100, 2)
            if ratio > 0:
                lang_ratio[lang_name] = ratio

    return lang_ratio

# 이 함수는 논란 댓글 여부를 판단할 때 사용하는 코드
# 각각의 댓글이 논란이 있는지 정상 댓글인지 판단함
def is_controversial(comment: str, threshold: float = 0.7) -> bool:
    """논란 댓글 여부 판단"""
    if not comment or len(comment.strip()) < 3:
        return False

    labels = ["논란 있음", "정상 댓글"]
    result = classifier(comment, candidate_labels=labels)
    return result["labels"][0] == "논란 있음" and result["scores"][0] >= threshold

# (위 함수 설명과 이어서)
# 이 댓글들이 전체 댓글 중 10% 이상이면 true값 리턴함
def is_video_controversial(comments: List[str], ratio_threshold: float = 0.1) -> bool:
    """영상 논란 여부 판단"""
    if not comments:
        return False

    suspicious_count = sum(1 for comment in comments if is_controversial(comment))
    return (suspicious_count / len(comments)) >= ratio_threshold

In [ ]:
# Fast api 코드
app = FastAPI(title="YouTube Comment Analyzer", version="1.0.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    return {"message": "YouTube Comment Analyzer API", "status": "running"}

@app.post("/analyze", response_model=AnalysisResponse)
@app.post("/analyze/", response_model=AnalysisResponse)
async def analyze(request: AnalysisRequest):
    print("/analyze 요청 도착")

    try:
        video_id = request.videoId
        comments_dict = request.comments

        if not comments_dict:
            raise HTTPException(status_code=400, detail="댓글 데이터가 없습니다.")

        comment_texts = list(comments_dict.values())
        summary = summarize_comments_with_gpt(comment_texts) # 전체 댓글 요약
        sentiment_comments, sentiment_ratio = await analyze_sentiment_async(comments_dict) # 댓글 감정 분석
        keywords = extract_keywords_tfidf(comment_texts, top_n=5) # 키워드 추출
        language_ratio = detect_languages(comment_texts) # 언어 비율 추출
        is_warning = is_video_controversial(comment_texts) # 논란 의심 분석

        response = AnalysisResponse(
            videoId=None,
            apiVideoId=video_id,
            summation=summary,
            isWarning=is_warning,
            keywords=keywords,
            sentimentComments=sentiment_comments,
            languageRatio=language_ratio,
            sentimentRatio=sentiment_ratio
        )

        return response

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"분석 중 오류가 발생했습니다: {str(e)}")

In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [2693]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔥 /analyze 요청 도착!


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.891 seconds.
DEBUG:jieba:Loading model cost 0.891 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


INFO:     3.36.105.229:0 - "POST /analyze HTTP/1.1" 200 OK
🔥 /analyze 요청 도착!
INFO:     3.36.105.229:0 - "POST /analyze HTTP/1.1" 200 OK
